# Random Walk Modeling

_Deadline 18.04.19_

During this seminar, we shall emulate the random walk of a knight on a chessboard. Then, we'll find a way to pick a random (uniformly chosen) spanning tree of a graph!

## 1/ A knight in the dark

Consider an $n \times n$ chessboard with a single knight on it. 

1. Construct a network with all knight's possible moves. In this network each node represents chessboard locations and an edge between two locations appears if knight is admitted to move from one to another.

2. Implement simulation of knight random walk on chessboard

    * Calculate average probability to visit chessboard locations
    * Calculate average recurrence time of a node
    
_But first, where could we go without our sanctified preamble?_

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import choice, rand 
import networkx as nx
%matplotlib inline

In [ ]:
def GenKnightNetwork(boardSize):
    G = nx.Graph()
    pos = {}
    for col in range(boardSize):
        for row in range(boardSize):
            nodeId = row + col*boardSize
            pos[nodeId] = np.array([1.0*row/boardSize, 1.0*col/boardSize])
            newPos = GetLegalMoves(row, col, boardSize)
            for e in newPos:
                nid = e[0] + e[1]*boardSize
                G.add_edge(nodeId, nid)
    return G, pos

def GetLegalMoves(x,y,boardSize):
    newMoves = []
    moveOffsets = [(-1,-2),(-1,2),(-2,-1),(-2,1),
                   ( 1,-2),( 1,2),( 2,-1),( 2,1)]
    for i in moveOffsets:
        newX = x + i[0]
        newY = y + i[1]
        if isLegalCoord(newX,boardSize) and isLegalCoord(newY,boardSize):
            newMoves.append((newX,newY))
    return newMoves

def isLegalCoord(x,boardSize):
    if x >= 0 and x < boardSize:
        return True
    else:
        return False

In [ ]:
boardSize = 8
(G,pos) = GenKnightNetwork(boardSize)
nx.draw_networkx(G,pos)

_Next, we shall implement the random walk in a finite graph. This next function return a list of visited nodes (ordered). Depending on the value of_ ```till_first_return```, _it runs for a specified number of steps or until it comes back to the first vertex (yes, it shall happen)._

In [ ]:
def RandomWalk(G, xi, n, till_first_return = False):
    nodeSeq = []
    nodeSeq.append(xi)
    if till_first_return:
        xInit = xi
        while True:
            xi = choice(list(G.neighbors(xi)),1)[0]       
            nodeSeq.append(xi)
            if xi == xInit:
                return nodeSeq
    else:
        for i in range(n):
            xi = choice(list(G.neighbors(xi)),1)[0]       
            nodeSeq.append(xi)
        return nodeSeq

Let us first walk around. Run this cell several time to see if behavior is homogeneous.

In [ ]:
nodeSeq = RandomWalk(G, 0, 100)
edgeSeq = [(nodeSeq[i-1], nodeSeq[i]) for i in range(1,len(nodeSeq))]
nx.draw(G,pos,edge_color='gray',width=.2)
nx.draw(G, pos, edgelist = edgeSeq, edge_color='blue', width=2)

Now let us walk until we come back home. Run this cell several time to see if behavior is homogeneous.

In [ ]:
nodeSeq = RandomWalk(G, 0, 100, True)
edgeSeq = [(nodeSeq[i-1], nodeSeq[i]) for i in range(1,len(nodeSeq))]
nx.draw(G,pos,edge_color='gray',width=.2)
nx.draw(G, pos, edgelist = edgeSeq, edge_color='blue', width=2)

Now, let us model a random walk starting from each node of the graph. We shall merge these sequences and observe statistics on this giant list.

### Task 1
_Start a random walk of length 1000 from each node of the graph and concatenate all these random walks. Then plot the histogram of this large sequence. Compare to the bar plot of degree distribution of $G$._

In [ ]:
## Build a giant sequence here.

In [ ]:
# Plot histogram.

In [ ]:
# The degree distribution of G.

a = plt.bar(range(64),[G.degree(i) for i in range(64)])

Finally, let us observe the number of steps needed to come back to the starting node (let us say node 27 here).

In [ ]:
returnTime = []
for i in range(1000):
    returnTime.append(len(RandomWalk(G, 27, 0, True)))

In [ ]:
a = plt.boxplot(returnTime)

### Task 2
_Does this time depends on the chosen starting node? Can you give any intuition? Plot the average return time from every vertex of the graph. Compare with degree distribution._

In [ ]:
# Code

Text

## 2 Generating spanning trees

Given a connected graph $G=(V,E)$, a spanning tree of $G$ is a tree $T=(V,F)$ with the same set of nodes as $G$ and such that its edges are a subset of those of $G$ ($F \subseteq E$). Any connected graph has a spanning tree. But there can be many (like exponentially many) of them.

### Question 3
_How many spanning trees for the graph $K_2$ (complete graph on two nodes), $K_3$, $K_4$? What about $K_n$?_

Your answer...

Taking a random spanning tree uniformly can be quite complicated for that matter. We don't want to list them all before picking one!

A pretty simple algorithm based on random walk gives a good generating model. It does the following:

```
WHILE every node has not been visited
    random walk one step
    IF new node is found THEN store the edge used for its discovery
RETURN the set of stored edges.
```

### Task 4

_Implement such an algorithm. Maintain a counter of how many random steps are taken._

In [ ]:
def gen_random_spanning_tree(G):
    # Your code here the next three lines are to be removed!
    counter = 0
    E_T = []
    return counter, E_T

## For your information (not to be implemented)

Still, discovering the whole graph can take some time for a random walk. In 1996, Wilson [1] provided a new algorithm which also produces a random (uniformly chosen) spanning tree. The tree grows as follows:

1. First pick a node $v$ and let $T$ be the tree consisting of only $v$ (one node, no edge).
2. While $T$ is not spanning (have not been reached yet)
    1. pick a node $u$ of $G$ not in $T$
    2. starting from $u$, walk randomly in $G$ until you hit some vertex of $T$.
    3. delete loops from this random walks.
    4. plug it to your current tree $T$.
3. Return $T$.

In a sense, this algorithms "prevents" to stick in the visited component for too long.


[1] Wilson D. B., Generating random spanning trees more quickly than the cover time, _Proceedings of the Twenty-eighth Annual ACM Symposium on the Theory of Computing (Philadelphia, PA, 1996)_, pp. 296-303, ACM, New York, 1996.

Let us plot the number of steps taken by your algorithm on several random graphs.

In [ ]:
sample = 10
p=.3
Tsimple=[]
for n in range(50,300,20):
    localTsimple=[]
    for i in range(sample):
        G = nx.erdos_renyi_graph(n,p)
        c,T = gen_random_spanning_tree(G)
        localTsimple.append(c)
    Tsimple.append(sum(localTsimple)/len(localTsimple))

In [ ]:
a=plt.plot(Tsimple)